<a href="https://www.kaggle.com/code/aliniazi3/spotcancerai?scriptVersionId=241933698" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# **Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
import os
import itertools
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.impute import SimpleImputer

from glob import glob
from PIL import Image
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

# **DATA READING**

In [ ]:
df = pd.read_csv('../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

# **DATA Preprocessing**

**Creating dictionary for displaying more human-friendly labels.**

In [ ]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}
base_skin_dir = '../input/skin-cancer-mnist-ham10000'

# **Merge images from both folders into one dictionary**

In [ ]:
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

# **Creating new columns for better understanding of features**

In [ ]:
df['path'] = df['image_id'].map(imageid_path_dict.get)
df['cell_type'] = df['dx'].map(lesion_type_dict.get)
df['cell_type_idx'] = pd.Categorical(df['cell_type']).codes

df.head()

In [ ]:
def preprocess_image(image_path, IMAGE_SIZE=456):  
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, cv2.getStructuringElement(cv2.MORPH_RECT, (17, 17)))
    inpainted = cv2.inpaint(img, blackhat, 1, cv2.INPAINT_TELEA)
    denoised = cv2.GaussianBlur(inpainted, (7, 7), 0)
    return denoised


# Drop missing paths
df = df.dropna(subset=['path'])

# Select one sample image per class
sample_images = df.groupby('cell_type').first().reset_index()

plt.figure(figsize=(14, 5))
for i, row in enumerate(sample_images.itertuples(), 1):
    if i > 7: break  # Limit to 7 images
    
    original_image = cv2.imread(row.path)
    processed_image = preprocess_image(row.path)
    
    original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    
    plt.subplot(2, 7, i)
    plt.imshow(original_image_rgb)
    plt.title(f"Original\n{row.cell_type}")
    plt.axis("off")
    
    plt.subplot(2, 7, i + 7)
    plt.imshow(processed_image, cmap="gray")
    plt.title("Processed")
    plt.axis("off")

plt.tight_layout()
plt.show()

df.head()

In [ ]:
tabular_data = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
tabular_data.head()

# **Exploratory Data Analysis (EDA)**

In [ ]:
tabular_data.columns

In [ ]:
classes = {
    0: ('akiec', 'Actinic keratoses and intraepithelial carcinomae'),
    1: ('bcc', ' basal cell carcinoma'),
    2: ('bkl', 'benign keratosis-like lesions'),
    3: ('df', 'dermatofibroma'),
    4: ('nv', ' melanocytic nevi'),
    5: ('vasc', ' pyogenic granulomas and hemorrhage'),
    6: ('mel', 'melanoma')
}


# **Frequency Distribution of Classes**

In [ ]:
sns.countplot(x = 'dx', data = tabular_data)
plt.xlabel('Disease', size=12)
plt.ylabel('Frequency', size=12)
plt.title('Frequency Distribution of Classes', size=16)
plt.show()

# **Distribution of Disease over Gender**

In [ ]:
bar, ax = plt.subplots(figsize = (10,10))
plt.pie(tabular_data['sex'].value_counts(), labels = tabular_data['sex'].value_counts().index, autopct="%.1f%%")
plt.title('Gender of Patient', size=16)
plt.show()

# **Histogram of Age of Patients**

In [ ]:
bar, ax = plt.subplots(figsize=(10,10))
sns.histplot(tabular_data['age'])
plt.title('Histogram of Age of Patients', size=16)
plt.show()

# **Location of disease over Gender**

In [ ]:
value = tabular_data[['localization', 'sex']].value_counts().to_frame()
value.reset_index(level=[1,0 ], inplace=True)
temp = value.rename(columns = {'localization':'location', 0: 'count'})

bar, ax = plt.subplots(figsize = (12, 12))
sns.barplot(x = 'location',  y='count', hue = 'sex', data = temp)
plt.title('Location of disease over Gender', size = 16)
plt.xlabel('Disease', size=12)
plt.ylabel('Frequency/Count', size=12)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into features (X) and labels (y)
X = df['path'].values  # Image file paths
y = df['cell_type_idx'].values  # Integer labels for the skin cancer types

# Split the dataset into train and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Print the size of the splits
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

# **Combine into DataFrames**

In [ ]:
# Create train and test DataFrames
train_df = pd.DataFrame({'path': X_train, 'label': y_train})
test_df = pd.DataFrame({'path': X_test, 'label': y_test})

# **Class Balance Using Resampling**

In [ ]:
class_counts = train_df['label'].value_counts()
max_count = class_counts.max()
print("Original class distribution:\n", class_counts)


In [ ]:
augmented_dir = "/kaggle/working/augmented_images"
os.makedirs(augmented_dir, exist_ok=True)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import uuid
IMAGE_SIZE= 456
augmenter = ImageDataGenerator(
    rotation_range=40,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

augmented_records = []

for label in class_counts.index:
    current_df = train_df[train_df['label'] == label]
    n_needed = max_count - len(current_df)
    
    if n_needed == 0:
        continue  # skip majority class
    
    print(f"Augmenting class '{label}' with {n_needed} new images...")
    
    i = 0
    for idx, row in current_df.iterrows():
        img = load_img(row['path'], target_size=(IMAGE_SIZE, IMAGE_SIZE))
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)
        
        # Generate augmented images
        for batch in augmenter.flow(x, batch_size=1):
            save_path = os.path.join(augmented_dir, f"{uuid.uuid4().hex}.jpg")
            array_to_img(batch[0]).save(save_path)
            augmented_records.append({'path': save_path, 'label': label})
            i += 1
            if i >= n_needed:
                break
        if i >= n_needed:
            break


In [ ]:
augmented_df = pd.DataFrame(augmented_records)
balanced_train_df = pd.concat([train_df, augmented_df], ignore_index=True)
balanced_train_df = balanced_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
plt.figure(figsize=(16, 6))

# --- Plot 1: Original Data Distribution ---
plt.subplot(1, 2, 1)
sns.countplot(data=train_df, x='label', order=train_df['label'].value_counts().index)
plt.title("Original Class Distribution")
plt.xlabel("Class Label")
plt.ylabel("Image Count")
plt.xticks(rotation=45)

# --- Plot 2: Balanced Data Distribution ---
plt.subplot(1, 2, 2)
sns.countplot(data=balanced_train_df, x='label', order=balanced_train_df['label'].value_counts().index)
plt.title("Balanced Class Distribution (Augmented, No Duplication)")
plt.xlabel("Class Label")
plt.ylabel("Image Count")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

# **One-hot encode the labels And Data Generators (for categorical classification)**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Ensure labels are string type (important for categorical labels)
balanced_train_df['label'] = balanced_train_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)

IMAGE_SIZE = 456  # EfficientNetB5 input size
BATCH_SIZE = 32
SEED = 42  # for reproducibility

# Data generators with augmentation for training, basic preprocessing for testing
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=40,
    zoom_range=0.3,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

# Use balanced_train_df here instead of train_df
train_generator = train_datagen.flow_from_dataframe(
    balanced_train_df,
    x_col='path',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)


test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='path',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED
)

plt.show()

# **Build and compile the EfficientNetB5 model**

In [ ]:
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define image size for EfficientNetB5
IMAGE_SIZE = 456  # EfficientNetB5 default input size
BATCH_SIZE = 32
# Load EfficientNetB5 without top layer
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Freeze all except last 20 layers
    layer.trainable = False
    
# Custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Build model
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# **Train the model**

In [ ]:
import warnings
warnings.filterwarnings("ignore")
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=15,
)

In [ ]:
# Example: Find epoch with the highest validation accuracy
best_epoch = np.argmax(history.history['accuracy']) + 1  # +1 because indexing starts from 0
best_acc = history.history['accuracy'][best_epoch - 1]

print(f"Best Epoch: {best_epoch}")
print(f"Accuracy at Best Epoch: {best_acc:.4f}")

In [ ]:
model.save('/kaggle/working/final_model.h5')

# **Plot Training History (Accuracy & Loss)**

In [ ]:
# Training curve
plt.figure(figsize=(14, 6))

# Loss curve
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss (training)')
plt.plot(history.history['val_loss'], label='Loss (validation)')
plt.title('Loss over the ages')
plt.xlabel('Eras')
plt.ylabel('Precision')
plt.legend()

# Precision curve
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy (training)')
plt.plot(history.history['val_accuracy'], label='Accuracy (validation)')
plt.title('Precision over the ages')
plt.xlabel('Eras')
plt.ylabel('Precision')
plt.legend()

plt.tight_layout()
plt.show()


# **Evaluate the Model on Test Data**

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# **Confusion Metrix**

In [ ]:
# Step 1: Get predictions from the model
y_prob = model.predict(test_generator)  # Probabilities
y_pred = np.argmax(y_prob, axis=1)      # Predicted labels

# Step 2: Get true labels from the test generator
y_true = test_generator.classes          # Actual labels

# Step 3: Get class names from the generator
CLASS_NAMES = list(test_generator.class_indices.keys())

# Step 4: Generate and plot confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Step 5: Classification report
print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))